Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [1]:
import requests

In [2]:
import time
time.sleep(0.2)

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
#KEYWORDS = ['python', 'парсинг'] #Ничего не находит, найдем другие слова
KEYWORDS = ['контекст', 'статья'] 

In [6]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [7]:
posts = soup.find_all('article', class_='post_preview')
# len(posts)
# posts

In [8]:
news_list = []
for post in posts:
    post_id = post.parent.attrs.get('id')
   
    if not post_id:
        continue
    
    post_id = int(post_id.split('_')[-1])
    text_preview = post.find_all('div', class_='post__text')
    
    for preview in text_preview:
#         print(preview)
        preview_lower = preview.text.lower()
#         print(preview_lower) 
        if any([desired in preview_lower for desired in KEYWORDS]):
#             print(preview_lower)
                
            date_element = post.find('span', class_ = 'post__time')
            title_element = post.find('a', class_='post__title_link')
            link_element = title_element.attrs.get('href')
           
            news_list.append({'date': date_element.text, 'title': title_element.text, 'link': link_element})
            
df = pd.DataFrame(news_list)
df

,date,title,link
0,сегодня в 19:01,Почему нам нужно больше Open Source Data Science,https://habr.com/ru/company/skillfactory/blog/...
1,сегодня в 18:34,Локализация в ASP.NET Core Razor Pages — Культуры,https://habr.com/ru/company/otus/blog/525136/
2,сегодня в 16:18,Что такое Waypoint и какие возможности дает ег...,https://habr.com/ru/company/southbridge/blog/5...
3,сегодня в 15:44,Как я делаю свой дом умным,https://habr.com/ru/post/525074/


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <да
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>


In [58]:
import json
import datetime
import time
time.sleep(0.2)
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {
    "emailAddresses":['xxxx@gmail.ru', 'yyyy@rambler.ru'],
}
headers = {
    'Vaar-Version':'0',
    'Vaar-Header-App-Product':'hackcheck-web-avast'
}
res = requests.post(url, headers=headers, data=json.dumps(params))
res.json()

breach_email = {}
df = pd.DataFrame()

for email, breaches in res.json()['summary'].items():
    breach_email[email] = list(breaches.values())[0]

for id_, details in res.json()['breaches'].items():
    for email, breaches in breach_email.items():
        if int(id_) in breaches:
            row = {'Почта': email, 
                   'Дата утечки': details['publishDate'], 
                   'Источник утечки': details['site'],
                   'Описание утечки': details['description']
                  }
            df = pd.concat([df, pd.DataFrame([row])])

df

,Почта,Дата утечки,Источник утечки,Описание утечки
0,xxxx@gmail.ru,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,yyyy@rambler.ru,2016-10-21T00:00:00Z,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,yyyy@rambler.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,yyyy@rambler.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxxx@gmail.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,yyyy@rambler.ru,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
0,yyyy@rambler.ru,2016-10-23T00:00:00Z,rambler.ru,"On February 17, 2012, Russian email provider R..."
0,yyyy@rambler.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,yyyy@rambler.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
